# Libraries

In [1]:
# libraries
# ----------

import requests
import pandas as pd
import wget

ModuleNotFoundError: ignored

# Downloading data

# Dataframes

In [ ]:
# dataset
# --------

conf_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recv_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [ ]:
conf_df.head()
# deaths_df.head()
# recv_df.head()

In [ ]:
#conf_df.columns
# deaths_df.columns
# recv_df.columns

In [ ]:
conf_df.columns[4:]

# Merging dataframes

In [ ]:
# extract dates
dates = conf_df.columns[4:]

# melt dataframes in longer format
conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

recv_df_long = recv_df_long[recv_df_long['Country/Region']!='Canada']

print(conf_df_long.shape)
print(deaths_df_long.shape)
print(recv_df_long.shape)

In [ ]:
# merge dataframes

# full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], 
#                        axis=1, sort=False)

full_table = pd.merge(left=conf_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recv_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

full_table.head()

In [ ]:
full_table.shape

In [ ]:
full_table.isna().sum()

In [ ]:
full_table[full_table['Recovered'].isna()]['Country/Region'].value_counts()

In [ ]:
full_table[full_table['Recovered'].isna()]['Date'].value_counts()

In [ ]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered'] = full_table['Recovered'].astype('int')
full_table.isna().sum()

# Preprocessing

In [ ]:
# renaming
# ========

# renaming countries, regions, provinces
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')

In [ ]:
# removing
# =======

# removing canada's recovered values
full_table = full_table[full_table['Province/State'].str.contains('Recovered')!=True]

# removing county wise data to avoid double counting
full_table = full_table[full_table['Province/State'].str.contains(',')!=True]

# Fixing off data

In [ ]:
# new values
feb_12_conf = {'Hubei' : 34874}

In [ ]:
# function to change value
def change_val(date, ref_col, val_col, dtnry):
    for key, val in dtnry.items():
        full_table.loc[(full_table['Date']==date) & (full_table[ref_col]==key), val_col] = val

In [ ]:
# changing values
change_val('2/12/20', 'Province/State', 'Confirmed', feb_12_conf)

In [ ]:
# checking values
full_table[(full_table['Date']=='2/12/20') & (full_table['Province/State']=='Hubei')]

# Saving final data

In [ ]:
full_table.to_csv('covid_19_clean_complete.csv', index=False)

# Country wise data

In [ ]:
# # china
# # =====

# china_province_wise = full_table[full_table['Country/Region']=='China']
# china_province_wise['Province/State'].unique()
# china_province_wise.to_csv('china_province_wise.csv', index=False)

In [ ]:
# # Australia
# # =========

# australia_state_wise = full_table[full_table['Country/Region']=='Australia']
# australia_state_wise['Province/State'].unique()
# australia_state_wise.to_csv('australia_state_wise.csv', index=False)

In [ ]:
# # Canada
# # ======

# canada_state_wise = full_table[full_table['Country/Region']=='Canada']
# canada_state_wise['Province/State'].unique()
# canada_state_wise.to_csv('canada_state_wise.csv', index=False)

# USA data

In [ ]:
us_conf_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
us_deaths_df = pd.read_csv('time_series_covid19_deaths_US.csv')

In [ ]:
#us_conf_df.head()
# us_deaths_df.head()

In [ ]:
# us_conf_df.columns
# us_deaths_df.columns

In [ ]:
ids = us_conf_df.columns[0:11]
us_dates = us_conf_df.columns[11:]

us_conf_df_long = us_conf_df.melt(id_vars=ids, value_vars=us_dates, var_name='Date', value_name='Confirmed')
us_deaths_df_long = us_deaths_df.melt(id_vars=ids, value_vars=us_dates, var_name='Date', value_name='Deaths')

In [ ]:
us_conf_df_long.head()
# us_deaths_df_long.head()

In [ ]:
print(us_conf_df_long.shape)
print(us_deaths_df_long.shape)

In [ ]:
ft_ids = us_conf_df_long.columns[:-1]
ft_ids

In [ ]:
us_full_table = pd.concat([us_conf_df_long, us_deaths_df_long[['Deaths']]], axis=1)
us_full_table.head()

In [ ]:
us_full_table.to_csv('usa_county_wise.csv', index=False)

# Tests data from https://www.worldometers.info/coronavirus/

In [ ]:
url = 'https://www.worldometers.info/coronavirus/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

In [ ]:
tests = pd.read_html(r.text)[0]
tests.head()

In [ ]:
tests.columns

In [ ]:
tests.columns = ['#', 'Country', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'ActiveCases', 'Serious,Critical', 'Cases per 1M pop',
       'Deaths per 1M pop', 'Total Tests', 'Tests per 1M pop', 'Population']

In [ ]:
tests = tests.loc[1:len(tests)-2, ['Country', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
                                   'TotalRecovered', 'ActiveCases', 'Serious,Critical', 
                                   'Cases per 1M pop', 'Deaths per 1M pop', 'Total Tests', 
                                   'Tests per 1M pop', 'Population']]

In [ ]:
# tests.columns = ['Country', 'Cases/1M Pop', 'Deaths/1M Pop', 'Total Tests', 'Tests/1M Pop']
tests

In [ ]:
tests.to_csv('tests.csv', index=False)